In [1]:
!pip install langchain lark chromadb pypdf google-cloud-aiplatform google-auth > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.


There are many PDFs and papers in below location, which will be used for embedding and further querying it.

https://github.com/insightbuilder/python_de_learners_data/tree/main/resources

In [114]:
#Loading the documents from langchain resources folder

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf(path_pdf):
  get_text = PyPDFLoader(path_pdf)
  
  get_pages = get_text.load()

  final_text = []

  shredder = RecursiveCharacterTextSplitter(chunk_size=350,
                                            chunk_overlap=20,
                                            length_function=len) 
  
  final_shred = shredder.split_documents(get_pages)

  return final_shred


In [6]:
#Just to test the function
agent_pg = load_pdf("/content/Agents.pdf")

In [7]:
agent_pg[0]

Document(page_content='5/31/23, 6:16 AM Agents — \x00\x00 LangChain 0.0.186\nhttps://python.langchain.com/en/stable/modules/agents.html 1/3Agents\nContents\nAction Agents', metadata={'source': '/content/Agents.pdf', 'page': 0})

In [115]:
import glob
file_list = glob.glob("/content/*.pdf")
all_docs = []
for file in file_list:
  temp_docs = load_pdf(file)
  all_docs.extend(temp_docs)

In [116]:
len(all_docs)

150

In [13]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/content/generativeaitrial-trialLC.json"

In [14]:
from langchain.schema import Document
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = VertexAIEmbeddings()

In [118]:
vectordb = ''

In [119]:
persist_directory = 'lc_documentdb'
vectordb = Chroma.from_documents(documents=all_docs,
                  persist_directory=persist_directory, 
                  embedding=embeddings)

In [120]:
vectordb.persist()

In [121]:
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)

In [ ]:
#vectordb.add_documents(all_docs[351:407])

In [78]:
#vectordb.persist()

In [122]:
db_retriever = vectordb.as_retriever()
db_retriever.get_relevant_documents("langchain concepts")

[Document(page_content='5/31/23, 6:13 AM Concepts — \x00\x00 LangChain 0.0.186\nhttps://python.langchain.com/en/stable/getting_started/concepts.html 1/3Concepts\nContents\nChain of Thought\nAction Plan Generation\nReAct\nSelf-ask\nPrompt Chaining\nMemetic Proxy\nSelf Consistency\nInception\nMemPrompt\nThese are concepts and terminology commonly used when developing LLM applications. It', metadata={'source': '/content/Concepts.pdf', 'page': 0}),
 Document(page_content='This is the Python specific portion of the documentation. For a purely conceptual guide to\nLangChain, see here. For the JavaScript documentation, see here.\nGetting Started\nHow to get started using LangChain to create an Language Model application.\nQuickstart Guide\nConcepts and terminology .\nConcepts and terminology', metadata={'source': '/content/WelcometoLangChain.pdf', 'page': 0}),
 Document(page_content='contains reference to external papers or sources where the concept was first introduced, as\nwell as to places

In [124]:
!zip -r lc_documentdb.zip /content/lc_documentdb

  adding: content/lc_documentdb/ (stored 0%)
  adding: content/lc_documentdb/chroma-embeddings.parquet (deflated 19%)
  adding: content/lc_documentdb/index/ (stored 0%)
  adding: content/lc_documentdb/index/uuid_to_id_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl (deflated 39%)
  adding: content/lc_documentdb/index/id_to_uuid_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl (deflated 34%)
  adding: content/lc_documentdb/index/index_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.bin (deflated 10%)
  adding: content/lc_documentdb/index/index_metadata_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl (deflated 14%)
  adding: content/lc_documentdb/chroma-collections.parquet (deflated 50%)


In [125]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info=[
    AttributeInfo(
        name="source",
        description="Filename and location of the source file", 
        type="string", 
    ),
    AttributeInfo(
        name="page",
        description="Page number on which the document is found", 
        type="integer", 
    )
]
document_content_description = "Text documents from Langchain help and concept documentation"

In [126]:
from langchain.llms import VertexAI

llm = VertexAI(temperature=0)

retriever = SelfQueryRetriever.from_llm(llm, 
                                        vectordb, 
                                        document_content_description, 
                                        metadata_field_info, 
                                        verbose=True,
                                        )

In [127]:
retriever.get_relevant_documents("What are some concepts of Agents")

query='Agents' filter=None limit=None


[Document(page_content='1. “Action Agents”: these agents decide an action to take and take that action one step at a\ntime\n2. “Plan-and-Execute Agents”: these agents first decide a plan of actions to take, and then\nexecute those actions one at a time.\nWhen should you use each one? Action Agents are more conventional, and good for small', metadata={'source': '/content/Agents.pdf', 'page': 0}),
 Document(page_content='Chains : Chains are structured sequences of calls (to an LLM or to a dif ferent utility).\nAgents : An agent is a Chain in which an LLM, given a high-level directive and a set of\ntools, repeatedly decides an action, executes the action and observes the outcome until\nthe high-level directive is complete.', metadata={'source': '/content/WelcometoLangChain.pdf', 'page': 1}),
 Document(page_content='5/31/23, 6:16 AM Agents — \x00\x00 LangChain 0.0.186\nhttps://python.langchain.com/en/stable/modules/agents.html 1/3Agents\nContents\nAction Agents\nPlan-and-Execute Agents\nCo

In [128]:
retriever = SelfQueryRetriever.from_llm(llm, 
                                        vectordb, 
                                        document_content_description, 
                                        metadata_field_info, 
                                        verbose=True,
                                        enable_limit=True)

In [130]:
retriever.get_relevant_documents("Explain 3 concepts of Chains")

query='explain 3 concepts of chains' filter=None limit=3


[Document(page_content='but understanding how it works will set you up well for working with more complex chains.\nFor more details, check out the getting started guide for chains.\nAgents: Dynamically Call Chains Based on\nUser Input\nSo far the chains we’ve looked at run in a predetermined order .', metadata={'source': '/content/QuickstartGuide.pdf', 'page': 4}),
 Document(page_content='Chains: Combine LLMs and prompts in multi-\nstep workflows\nUp until now , we’ve worked with the PromptT emplate and LLM primitives by themselves. But of\ncourse, a real application is not just one primitive, but rather a combination of them.\nA chain in LangChain is made up of links, which can be either primitives like LLMs or other\nchains.', metadata={'source': '/content/QuickstartGuide.pdf', 'page': 3}),
 Document(page_content='5/31/23, 6:13 AM Concepts — \x00\x00 LangChain 0.0.186\nhttps://python.langchain.com/en/stable/getting_started/concepts.html 1/3Concepts\nContents\nChain of Thought\nAction

In [131]:
retriever.get_relevant_documents("Give 2 example of autonomous agent")

query='autonomous agent' filter=None limit=2


[Document(page_content='The most important abstraction of the four above to understand is that of the agent. Although\nan agent can be defined in whatever way one chooses, the typical way to construct an agent is\nwith:\nPromptT emplate: this is responsible for taking the user input and previous steps and\nconstructing a prompt to send to the language model', metadata={'source': '/content/Agents.pdf', 'page': 1}),
 Document(page_content='potentially an unknown chain that depends on the user ’s input. In these types of chains, there\nis a “agent” which has access to a suite of tools. Depending on the user input, the agent can\nthen decide which, if any , of these tools to call.\nAt the moment, there are two main types of agents:', metadata={'source': '/content/Agents.pdf', 'page': 0})]

In [132]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(llm, 
                                                    chain_type="stuff", 
                                                    retriever=retriever)

In [135]:
chain({"question":"Give 2 types of agents"})

query='types of agents' filter=None limit=2


{'question': 'Give 2 types of agents',
 'answer': '1. “Action Agents”: these agents decide an action to take and take that action one step at a\ntime\n2. “Plan-and-Execute Agents”: these agents first decide a plan of actions to take, and then\nexecute those actions one at a time.\n',
 'sources': '/content/Agents.pdf'}

In [136]:
chain({"question":"Give 2 examples of agents"})

query='agent' filter=None limit=2


{'question': 'Give 2 examples of agents',
 'answer': 'The two main types of agents are:\n1. Sequential agent\n2. Random agent\n',
 'sources': '/content/Agents.pdf'}

In [137]:
chain({"question":"How to combine LLMs"},
      return_only_outputs=False)

query='combine LLMs' filter=None limit=None


{'question': 'How to combine LLMs',
 'answer': 'To combine LLMs, you can use the LangChain library. A chain in LangChain is made up of links, which can be either primitives like LLMs or other chains.\n',
 'sources': '/content/QuickstartGuide.pdf'}

In [138]:
chain({"question":"Provide 2 examples of combining LLMs"},
      return_only_outputs=False)

query='combine LLMs' filter=None limit=2


{'question': 'Provide 2 examples of combining LLMs',
 'answer': 'Prompt Chaining and Memetic Proxy\n',
 'sources': '/content/Concepts.pdf'}

In [140]:
chain({"question":"4 Concepts in langchain"},
      return_only_outputs=False)

query='4 Concepts in langchain' filter=None limit=4


{'question': '4 Concepts in langchain',
 'answer': 'Chain of Thought, Action Plan Generation, ReAct, Self-ask\n',
 'sources': '/content/Concepts.pdf'}

In [141]:
chain({"question":"Explain consistency in langchain."},
      return_only_outputs=False)

query='explain consistency in langchain' filter=None limit=None


{'question': 'Explain consistency in langchain.',
 'answer': 'Self Consistency is a decoding strategy that samples a diverse set of reasoning paths and then selects the most consistent answer. Is most effective when combined with Chain-of-thought prompting.\n',
 'sources': '/content/Concepts.pdf'}